# Evaluating essentia key extractor over Beatport EDM datasets.

In this colabolatory notebook we are going going to show how to load Beatport EDM tempo dataset. And we are going to evalute the essentia key extractor `KeyExtractor(...)`.


### Importing mir_eval

In [1]:
!pip install git+https://github.com/PRamoneda/mir_eval.git@Pedro/only_key
import mir_eval


  Cloning https://github.com/PRamoneda/mir_eval.git (to revision Pedro/only_key) to /tmp/pip-req-build-lf8j6jv7
  Running command git clone -q https://github.com/PRamoneda/mir_eval.git /tmp/pip-req-build-lf8j6jv7
  Running command git checkout -b Pedro/only_key --track origin/Pedro/only_key
  Switched to a new branch 'Pedro/only_key'
  Branch 'Pedro/only_key' set up to track remote branch 'Pedro/only_key' from 'origin'.
  Created wheel for mir-eval: filename=mir_eval-0.6-cp36-none-any.whl size=97582 sha256=b0f706dba94042c2dcf61815832da15f52a2d543077e2428d956e79764207262
  Stored in directory: /tmp/pip-ephem-wheel-cache-cdic8l0g/wheels/0e/26/dd/6dc42f9f128108b06932044ac1f78f18635e85593f59bfefc5
Successfully built mir-eval


### Importing mirdata and loading Beatport key dataset


In [2]:
!pip install git+https://github.com/mir-dataset-loaders/mirdata.git@Pedro/bug/beatport_key

  Cloning https://github.com/mir-dataset-loaders/mirdata.git (to revision Pedro/bug/beatport_key) to /tmp/pip-req-build-9xdyjapg
  Running command git clone -q https://github.com/mir-dataset-loaders/mirdata.git /tmp/pip-req-build-9xdyjapg
  Running command git checkout -b Pedro/bug/beatport_key --track origin/Pedro/bug/beatport_key
  Switched to a new branch 'Pedro/bug/beatport_key'
  Branch 'Pedro/bug/beatport_key' set up to track remote branch 'Pedro/bug/beatport_key' from 'origin'.
     |████████████████████████████████| 184kB 2.6MB/s 
     |████████████████████████████████| 61kB 3.8MB/s 
     |████████████████████████████████| 5.6MB 4.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for mirdata: filename=mirdata-0.2.2-cp36-none-any.whl size=2586562 sha256=703a406760056814db52d37989ff7eb10b686914f6f3a839208323f63ce71a57
  Stored in directory: /tmp/p

In [3]:
import mirdata.beatport_key as bp_key

bp_key.download()

0.00B [00:00, ?B/s]

Starting to download ['keys', 'metadata', 'audio'] to folder /root/mir_datasets/beatport_key
> downloading keys


376kB [00:02, 145kB/s]                            
0.00B [00:00, ?B/s]

> downloading metadata


904kB [00:01, 556kB/s]                            
0.00B [00:00, ?B/s]

> downloading audio


1.98GB [04:32, 7.82MB/s]                            


In [7]:
bp_key.validate()

Success: the dataset is complete and all files are valid.
--------------------


({}, {})

### Importing essentia

In [8]:
!pip install essentia


In [9]:
from essentia.standard import KeyExtractor, MonoLoader


In [16]:
results = []
key_extractor = KeyExtractor()
for k, v in bp_key.load().items():
  # compute estimated key
  loader = MonoLoader(filename=v.audio_path)
  audio = loader()
  estimated_key = key_extractor(audio)
  estimated_key = estimated_key[0] + ' ' + estimated_key[1]
  # copmpute the best score of the possible keys
  score = []
  for r_key in v.key:
    if r_key.split(' ')[0].lower() == 'x': # if is ambiguous
        reference_key = 'X'
    else:
        reference_key = r_key.split(' ')[0] + ' ' + r_key.split(' ')[1]
    # _ and ^ not supported by mir_eval and essentia
    if not '^' in reference_key and not '_' in reference_key:
      score.append(mir_eval.key.weighted_score(reference_key, estimated_key))
  # add to results and print
  if not '^' in reference_key and not '_' in reference_key:
    results.append(str(max(score)))
  print("Reference key:", v.key, "Estimated key:", estimated_key, score)


Reference key: ['D minor'] Estimated key: D minor [1.0]
Reference key: ['F# minor'] Estimated key: D minor [0.0]
Reference key: ['D minor'] Estimated key: D minor [1.0]
Reference key: ['G# minor'] Estimated key: D minor [0.0]
Reference key: ['C minor'] Estimated key: D minor [0.0]
Reference key: ['D minor'] Estimated key: D minor [1.0]
Reference key: ['D# minor'] Estimated key: D minor [0.0]
Reference key: ['E major'] Estimated key: D minor [0.0]
Reference key: ['G# minor'] Estimated key: D minor [0.0]
Reference key: ['E minor', 'E major'] Estimated key: D minor [0.0, 0.0]
Reference key: ['F# minor'] Estimated key: D minor [0.0]
Reference key: ['C# major'] Estimated key: D minor [0.0]
Reference key: ['B minor'] Estimated key: D minor [0.0]
Reference key: ['D major'] Estimated key: D minor [0.2]
Reference key: ['D major'] Estimated key: D minor [0.2]
Reference key: ['G# major'] Estimated key: D minor [0.0]
Reference key: ['X'] Estimated key: D minor [0.0]
Reference key: ['C# minor'] Est

In [17]:
from collections import Counter
num_tracks = len(results)
bp_key_results = Counter(results)
print(num_tracks, bp_key_results)

1484 Counter({'0.0': 1230, '1.0': 92, '0.5': 81, '0.2': 49, '0.3': 32})


In [18]:
print("RESULTS:")
print("Same key: " + str(bp_key_results['1.0']/num_tracks*100) + "%")
print("Estimated key is a perfect fifth above reference key: " + str(bp_key_results['0.5']/num_tracks*100) + "%")
print("Relative major/minor: " + str(bp_key_results['0.3']/num_tracks*100) + "%")
print("Parallel major/minor: " + str(bp_key_results['0.2']/num_tracks*100) + "%")
print("Other: " + str(bp_key_results['0.0']/num_tracks*100) + "%")


RESULTS:
Same key: 6.199460916442049%
Estimated key is a perfect fifth above reference key: 5.45822102425876%
Relative major/minor: 2.15633423180593%
Parallel major/minor: 3.30188679245283%
Other: 82.88409703504043%
